In [1]:
import numpy as np
import math
from evtk.hl import imageToVTK
import nrrd
from scipy.stats import iqr
import vtk
from vtk.util import numpy_support
from vtkmodules.vtkCommonDataModel import vtkStructuredPoints

In [2]:
#Tangle function

x = np.linspace(-5, 5, num=64)
y = np.linspace(-5, 5, num=64)
z = np.linspace(-5, 5, num=64)

XX, YY, ZZ = np.meshgrid(x, y, z)

f = XX**4 - 5*XX**2 + YY**4 - 5*YY**2 + ZZ**4 - 5*ZZ**2 + 40

In [3]:
# Write ground truth
imageToVTK("./truth_tangle", pointData = {"tangle" : f})

'/Users/tm8/tusharathawale/research/projects/ECRP/inverseLinearInterpolationUncertaintyGaussian/code/python-code/prepare-data/tangle-data/output-usedby-other-code/recreate-TVCG-tangle-results/truth_tangle.vti'

In [3]:
# Perfectly correlated ensemble (i.e., correlation=1) with outlier members
def getCorrelatedEnsemble(data, numMembers, numoutliers):
    
    h,w,d = data.shape
    # 1 extra size to store variance / or kernel bandwidth
    ensemble = np.zeros((numMembers+1,h, w, d))
    
    np.random.seed(1)
    
    for i in range(numMembers - numOutliers):
        
        
        factor = np.random.uniform(0.99,1.01,1)
        
        ensemble[i,:,:,:] = factor*data
        
        
    for i in range(numMembers - numOutliers, numMembers):
        
        factor = np.random.uniform(1.03,1.07,1)
    
        ensemble[i,:,:,:] = factor*data
        
    return ensemble

In [4]:
# Generate the ensemble
numMems = 5
numOutliers = 1
ensemble_correlated = getCorrelatedEnsemble(f, numMems, numOutliers)

In [5]:
# Functions to write out numpy array to vtk files

def writeStructuredDs(fname, ds):
    writer = vtk.vtkStructuredPointsWriter()
    writer.SetFileName(fname)
    writer.SetFileVersion(42)
    writer.SetInputData(ds)
    writer.Update()
    writer.Write() 


# Write data with one field
def writeOneField(fname, fieldname1, data1):
    
    a,b,c = data1.shape
    
    xdim=a
    ydim=b
    zdim=c

    x,y,z = np.meshgrid(np.linspace(0,a,xdim), np.linspace(0,b,ydim), np.linspace(0,c,zdim), indexing='ij')
    print(x.size, y.size, z.size)

    structured_dataset = vtkStructuredPoints()
    structured_dataset.SetDimensions(xdim, ydim, zdim)
    structured_dataset.SetOrigin(0, 0, 0)
    #print(np.array(g).shape)
    
    vtkArray1 = numpy_support.numpy_to_vtk(np.array(data1).flatten())
    vtkArray1.SetNumberOfComponents(1)
    vtkArray1.SetName(fieldname1)

    structured_dataset.GetPointData().AddArray(vtkArray1)
    structured_dataset.GetPointData().SetActiveScalars(fieldname1)

    writeStructuredDs(fname,structured_dataset)

# Write data with two fields
# e.g.,writeTwoFields("uncertaintyBiVariateField.vtk", "minField", minimumCurlZ, "maxField", maximumCurlZ)
def writeTwoFields(fname, fieldname1, data1, fieldname2, data2):
    
    a,b,c = data1.shape
    
    xdim=a
    ydim=b
    zdim=c

    x,y,z = np.meshgrid(np.linspace(0,a,xdim), np.linspace(0,b,ydim), np.linspace(0,c,zdim), indexing='ij')
    print(x.size, y.size, z.size)

    structured_dataset = vtkStructuredPoints()
    structured_dataset.SetDimensions(xdim, ydim, zdim)
    structured_dataset.SetOrigin(0, 0, 0)
    #print(np.array(g).shape)
    
    vtkArray1 = numpy_support.numpy_to_vtk(np.array(data1).flatten())
    vtkArray1.SetNumberOfComponents(1)
    vtkArray1.SetName(fieldname1)
    
    vtkArray2 = numpy_support.numpy_to_vtk(np.array(data2).flatten())
    vtkArray2.SetNumberOfComponents(1)
    vtkArray2.SetName(fieldname2)


    structured_dataset.GetPointData().AddArray(vtkArray1)
    structured_dataset.GetPointData().SetActiveScalars(fieldname1)
    
    structured_dataset.GetPointData().AddArray(vtkArray2)
    structured_dataset.GetPointData().SetActiveScalars(fieldname2)

    writeStructuredDs(fname,structured_dataset)
    
# Write data with five fields
def writeFiveFields(fname, fieldname1, data1, fieldname2, data2, fieldname3, data3, fieldname4, data4, fieldname5, data5):
    
    a,b,c = data1.shape
    
    xdim=a
    ydim=b
    zdim=c

    x,y,z = np.meshgrid(np.linspace(0,a,xdim), np.linspace(0,b,ydim), np.linspace(0,c,zdim), indexing='ij')
    print(x.size, y.size, z.size)

    structured_dataset = vtkStructuredPoints()
    structured_dataset.SetDimensions(xdim, ydim, zdim)
    structured_dataset.SetOrigin(0, 0, 0)
    #print(np.array(g).shape)
    
    vtkArray1 = numpy_support.numpy_to_vtk(np.array(data1).flatten())
    vtkArray1.SetNumberOfComponents(1)
    vtkArray1.SetName(fieldname1)
    
    vtkArray2 = numpy_support.numpy_to_vtk(np.array(data2).flatten())
    vtkArray2.SetNumberOfComponents(1)
    vtkArray2.SetName(fieldname2)
    
    vtkArray3 = numpy_support.numpy_to_vtk(np.array(data3).flatten())
    vtkArray3.SetNumberOfComponents(1)
    vtkArray3.SetName(fieldname3)
    
    vtkArray4 = numpy_support.numpy_to_vtk(np.array(data4).flatten())
    vtkArray4.SetNumberOfComponents(1)
    vtkArray4.SetName(fieldname4)

    vtkArray5 = numpy_support.numpy_to_vtk(np.array(data5).flatten())
    vtkArray5.SetNumberOfComponents(1)
    vtkArray5.SetName(fieldname5)

    structured_dataset.GetPointData().AddArray(vtkArray1)
    structured_dataset.GetPointData().SetActiveScalars(fieldname1)
    
    structured_dataset.GetPointData().AddArray(vtkArray2)
    structured_dataset.GetPointData().SetActiveScalars(fieldname2)

    structured_dataset.GetPointData().AddArray(vtkArray3)
    structured_dataset.GetPointData().SetActiveScalars(fieldname3)
    
    structured_dataset.GetPointData().AddArray(vtkArray4)
    structured_dataset.GetPointData().SetActiveScalars(fieldname4)
    
    structured_dataset.GetPointData().AddArray(vtkArray5)
    structured_dataset.GetPointData().SetActiveScalars(fieldname5)
    
    writeStructuredDs(fname,structured_dataset)

In [11]:
# Data for the independent Gaussian model.
meanVol = np.mean(ensemble_correlated[0:numMems,:,:,:], axis=0)
varVol = np.var(ensemble_correlated[0:numMems,:,:,:], axis=0)
writeTwoFields("./tangle-ignore-correlation.vtk", "mean", meanVol, "variance", varVol)

262144 262144 262144


In [6]:
# Data for the independent uniform model.
minVol = np.amin(ensemble_correlated[0:numMems,:,:,:], axis=0)
maxVol = np.amax(ensemble_correlated[0:numMems,:,:,:], axis=0)

meanUniformVol = (minVol+maxVol)/2
widthUniformVol = maxVol - meanUniformVol
writeTwoFields("./tangle-independent-uniform.vtk", "mean", meanUniformVol, "variance", widthUniformVol)

262144 262144 262144


In [15]:
# Data for the Gaussian model with spatial correlation. Pack mean, variance, rhoX, rhoY, and rhoZ fields into a single vtk file

numMems = 6

meanVol = np.mean(ensemble_correlated[0:numMems,:,:,:], axis=0)

# Compute correlation in X, Y, and Z directions
rhoX = np.zeros((64,64,64))
rhoY = np.zeros((64,64,64))
rhoZ = np.zeros((64,64,64))
varVol = np.zeros((64,64,64))

for i in range(63):
    for j in range(63):
        for k in range(63):
            
            #****** np.var and np.covar can lead to slightly different variance value computation

            # Compute covariance in x direction
            x1 = ensemble_correlated[0:numMems,i,j,k]
            x2 = ensemble_correlated[0:numMems,i+1,j,k]
            X = np.stack((x1, x2), axis=0)
            rhoXMat = np.cov(X)
            rhoX[i,j,k] = rhoXMat[0,1]
            
            # Compute covariance in y direction
            y1 = ensemble_correlated[0:numMems,i,j,k]
            y2 = ensemble_correlated[0:numMems,i,j+1,k]
            Y = np.stack((y1, y2), axis=0)
            rhoYMat = np.cov(Y)
            rhoY[i,j,k] = rhoYMat[0,1]
            
            # Compute covariance in z direction
            z1 = ensemble_correlated[0:numMems,i,j,k]
            z2 = ensemble_correlated[0:numMems,i,j,k+1]
            Z = np.stack((z1, z2), axis=0)
            rhoZMat = np.cov(Z)
            rhoZ[i,j,k] = rhoZMat[0,1]
            
            varVol[i,j,k] = rhoXMat[0,0]
            
            if(i==62):
                varVol[i+1,j,k] = np.var(ensemble_correlated[0:numMems,i+1,j,k], axis=0)
            if(j==62):
                varVol[i,j+1,k] = np.var(ensemble_correlated[0:numMems,i,j+1,k], axis=0) 
            if(k==62):
                varVol[i,j,k+1] = np.var(ensemble_correlated[0:numMems,i,j,k+1], axis=0)    
                

writeFiveFields("./14-mems-tangle-consider-correlation.vtk", "mean", meanVol, "variance", varVol, "rhoX", rhoX, "rhoY", rhoY, "rhoZ", rhoZ)

262144 262144 262144


In [15]:
# Save the data for kernel density estimation (KDE) as individual vtk ensmeble members for viskores code
for i in range(numMems):
        ensembleMember = np.squeeze(ensemble_correlated[i,:,:,:])
        writeOneField('./tangle_correlation_one_'+str(i)+'.vtk', 'tangle', ensembleMember)

262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
262144 262144 262144
